In [2]:
from lusidtools.jupyter_tools import toggle_code

"""Luminesce Syntax

Attributes
----------

"""

toggle_code("Toggle Docstring")

*Setting up the environment for the tutorial*

This section of the notebook creates a portfolio, some equities, and a file in drive to run luminesce queries against.

In [ ]:
# install lusid-drive-sdk
!pip install lusid-drive-sdk

In [42]:
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
import lusid as lu
import pandas as pd
import lusid_drive
import lusid_drive.rest
import lusid.rest
from lusid_drive.rest import ApiException
from pprint import pprint
import json


secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

# Confirm success by printing SDK version
api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)
display(api_status)


,api_version,build_version,excel_version,links
0,v0,0.6.10250.0,0.5.3075,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [4]:
# here's some sample equities we've prepared earlier
ftse_instruments = pd.read_csv("_data/ftse100.csv")
nasdaq_instruments = pd.read_csv("_data/nasdaq100.csv")


In [44]:
# don't show exception if error is due to upsert
def exception_guard(e, code):
    return (
        e.status
        and e.status != "400 Bad Request"
        and e.body
        and json.loads(e.body)["code"] == code
    )


# create portfolio for this module
def upsert_portfolio(api_factory, scope, code, portfolio_name, portfolio_description):
    api_instance = api_factory.build(lu.TransactionPortfoliosApi)
    # Create an instance of the API class
    create_transaction_portfolio_request = {
        "displayName": portfolio_name,
        "description": portfolio_description,
        "code": code,
        "baseCurrency": "GBP",
        "instrumentScopes": ["FBNUniversityModule-T2-2"],
    }  # CreateTransactionPortfolioRequest | The definition of the transaction portfolio.
    try:
        # UpdatePortfolio: Update portfolio
        api_response = api_instance.create_portfolio(
            scope, create_transaction_portfolio_request
        )
        pprint(api_response)
    except lusid.rest.ApiException as e:
        if not exception_guard(e, 112):
            print("Exception when calling PortfoliosApi->update_portfolio: %s\n" % e)


scope = "FBNUniversity"  # str | The scope of the portfolio.
code = "Module-T2-2"  # str | The code of the portfolio. Together with the scope this uniquely identifies the portfolio.
portfolio_name = "Training module T2.2"
portfolio_description = "A portfolio to hold transactions"
upsert_portfolio(api_factory, scope, code, portfolio_name, portfolio_description)


In [6]:
# create a request to upsert a bunch of instruments.
def generate_upsert_instrument_request(
    api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier
):
    request_id = name + isin
    request = {
        "name": name,
        "identifiers": {
            "Figi": {"value": figi, "effectiveAt": "0001-01-01T00:00:00.0000000+00:00"},
            "Isin": {"value": isin, "effectiveAt": "0001-01-01T00:00:00.0000000+00:00"},
            other_identifier_name: {
                "value": other_identifier,
                "effectiveAt": "0001-01-01T00:00:00.0000000+00:00",
            },
        },
        "definition": {"instrumentType": "Equity", "domCcy": ccy},
    }
    return request_id, request


In [45]:
# Create an instance of the API class
api_instance = api_factory.build(lu.InstrumentsApi)

# We're gonna load some american and UK equities into LUSID

other_identifier_name = "Sedol"
ccy = "GBP"
upsert_body_tuple = (
    generate_upsert_instrument_request(
        api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier
    )
    for index, ticker, name, _, isin, other_identifier, figi in ftse_instruments.itertuples()
)
ftse_body_dict = {
    request_id: request_body for request_id, request_body in upsert_body_tuple
}
other_identifier_name = "Cusip"
ccy = "USD"
upsert_body_tuple = (
    generate_upsert_instrument_request(
        api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier
    )
    for index, ticker, name, _, isin, other_identifier, figi in nasdaq_instruments.itertuples()
)
nasdaq_body_dict = {
    request_id: request_body for request_id, request_body in upsert_body_tuple
}

request_body_dict = nasdaq_body_dict | ftse_body_dict

try:
    # UpsertInstruments: Upsert instruments
    api_response = api_instance.upsert_instruments(request_body_dict, scope=scope)
#     pprint(api_response)
except lusid.rest.ApiException as e:
    print("Exception when calling InstrumentsApi->upsert_instruments: %s\n" % e)


In [51]:
# let's chuck a file containing a randomly selected subset of 25 instruments with their sectors into drive for some examples:

subset_file = open("_data/ftse_subset.csv").read()

drive_config = lusid_drive.Configuration(host="https://fbn-uni.lusid.com/drive")
drive_config.access_token = RefreshingToken()

# Enter a context with an instance of the API client
with lusid_drive.ApiClient(drive_config) as api_client:
    # Create an instance of the API class
    api_instance = lusid_drive.FoldersApi(api_client)
    create_folder = {
        "path": "/",
        "name": "finbourne university",
    }  # CreateFolder | A CreateFolder object that defines the name and path of the new folder

    try:
        # [EARLY ACCESS] CreateFolder: Create a new folder in LUSID Drive
        api_response = api_instance.create_folder(create_folder)
        pprint(api_response)
    except ApiException as e:
        if not exception_guard(e, 664):
            print("Exception when calling FilesApi->create_file: %s\n" % e)

    api_instance = lusid_drive.FilesApi(api_client)
    x_lusid_drive_filename = "ftse_subset.csv"  # str | File name.
    x_lusid_drive_path = "/finbourne university/"  # str | File path.
    content_length = len(subset_file.encode("UTF-8"))
    body = subset_file  # str |

    try:
        # [EARLY ACCESS] CreateFile: Uploads a file to Lusid Drive. If using an SDK, consider using the UploadAsStreamAsync function for larger files instead.
        api_response = api_instance.create_file(
            x_lusid_drive_filename, x_lusid_drive_path, content_length, body
        )
    except lusid_drive.rest.ApiException as e:
        if not exception_guard(e, 671):
            print("Exception when calling FilesApi->create_file: %s\n" % e)


# Luminesce Syntax
## What is Structured Query Language (SQL)
SQL - pronounced "sequel" was first developed at IBM in the early 1970s, as a way to query and manipulate structured relational databases. SQL is a set-based, declarative programming language, however many implementations of SQL add extensions with procedural programming functionality, such as control-of-flow contructs.
## Luminesce Queries - SQL with extensions
SQLite is the most widely deployed database engine, used by several of the top web browsers, operating systems and application software.

Luminesce supports most of the query syntax for the SQLite dialect of SQL, but not the data manipulation or data definition syntax. Data can still be manipulated using some extensions to the language that Luminesce adds. 

Luminesce adds extensions to the SQLite implementation which can:
- exclude columns from queries
- automatically select important columns in queries
- parameterise queries
- write data to some Luminesce providers
- wait on results of some dependent operation, or for a time period
- run queries in parallel on an iterated range of parameters.

## Use case: Using Luminesce to query and join data from disparate sources, and to distribute the resulting information.
I'd like to read a file from Drive and use the data from this file to filter transaction results. I'd like to join these filtered results with another provider and distribute the results. This will be packaged up into a simple to use View which runs on a schedule.

## An intro to SQLite queries

### A query using a simple SELECT statement
Its simple to get data from a provider using a select statement.

Use the following statement to get all data from a provider:

`SELECT * FROM SOME.PROVIDER`

Or retrieve data from a subset of columns by specifying them:

`SELECT A_COLUMN, ANOTHER_COLUMN FROM SOME.PROVIDER`

Here we'll use a SELECT statement to query for all of our porfolios.

In [9]:
%%luminesce
SELECT * FROM Lusid.Portfolio

,PortfolioScope,PortfolioCode,PortfolioType,BaseCurrency,DisplayName,Description,OriginPortfolioScope,OriginPortfolioCode,ParentPortfolioScope,ParentPortfolioCode,ChildPortfolioIds,CorporateActionSourceScope,CorporateActionSourceId,AccountingMethod,SubHoldingKeys,Created,InstrumentScopes
0,FBNUniversity,Module-3-2,Transaction,GBP,Training module 3.2,NaN,FBNUniversity,Module-3-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-3-2
1,FBNUniversity,Module-3-1,Transaction,GBP,Training module 3.1,NaN,FBNUniversity,Module-3-1,NaN,NaN,NaN,NaN,NaN,Default,Transaction/FBNUniversity/Module-3-1,2022-01-01,FBNUniversityModule-3-1
2,FBNUniversity,Module-6-4,Transaction,GBP,Training module 6.4 - Track commission as sepa...,NaN,FBNUniversity,Module-6-4,NaN,NaN,NaN,NaN,NaN,Default,Transaction/FBNUniversity/TrackCommission,2022-01-01,FBNUniversityModule-6-4
3,FBNUniversity,Module-6-3,Transaction,GBP,Training module 6.3 - Reduce cash balance by c...,NaN,FBNUniversity,Module-6-3,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-3
4,FBNUniversity,Module-6-1,Transaction,GBP,Training module 6.1 - Different transaction co...,NaN,FBNUniversity,Module-6-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-1
5,FBNUniversity,Module-6-2,Transaction,GBP,"Training module 6.2 - Same transaction code, d...",NaN,FBNUniversity,Module-6-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-2
6,FBNUniversity,Module-T2-2,Transaction,GBP,Training module T2.2,A portfolio to hold transactions,FBNUniversity,Module-T2-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-11-03 20:14:15.334,FBNUniversityModule-T2-2
7,FBNUniversity,Module-1,Transaction,GBP,Module 1,NaN,FBNUniversity,Module-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2021-12-01,FBNUniversityModule-1
8,FBNUniversity,Module-4-1,Transaction,GBP,Training module 4.1,NaN,FBNUniversity,Module-4-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-4-1


We can specify column names if we are only interested in some parts of the data - here I really just want to see the name of the portfolio and the portfolio code:

In [10]:
%%luminesce
SELECT DisplayName, PortfolioCode FROM Lusid.Portfolio

,DisplayName,PortfolioCode
0,Training module 6.4 - Track commission as sepa...,Module-6-4
1,Module 1,Module-1
2,Training module 4.1,Module-4-1
3,Training module T2.2,Module-T2-2
4,Training module 3.1,Module-3-1
5,Training module 3.2,Module-3-2
6,Training module 6.3 - Reduce cash balance by c...,Module-6-3
7,Training module 6.1 - Different transaction co...,Module-6-1
8,"Training module 6.2 - Same transaction code, d...",Module-6-2


### Limiting the number of return results using LIMIT
These simple SELECT statements are useful for getting an idea of the shape and quality of the data returned by a provider, but they tend to return a lot of data. Lets use LIMIT to reduce the amount of data requested:

In [11]:
%%luminesce
SELECT DisplayName, PortfolioCode FROM Lusid.Portfolio LIMIT 5

,DisplayName,PortfolioCode
0,Training module 3.2,Module-3-2
1,Training module 3.1,Module-3-1
2,Training module 6.4 - Track commission as sepa...,Module-6-4
3,Training module 6.3 - Reduce cash balance by c...,Module-6-3
4,Training module 6.1 - Different transaction co...,Module-6-1


### Filter queries WHERE ...
We can now filter these results down - lets get all of the Equities that are listed in GBP


In [12]:
%%luminesce
SELECT * FROM Lusid.Instrument.Equity WHERE DomCcy = 'GBP'

,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,ClientInternal,Figi,CompositeFigi,ShareClassFigi,Wertpapier,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,LUID_00003D5K,NaN,NaN,NaN,NaN,NaN,BBG000C05BD1,NaN,NaN,NaN,...,FBNUniversityModule-3-1,BP,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
1,LUID_00003DE8,GB00B0LCW083,B0LCW08,NaN,NaN,NaN,BBG000FG1MF1,NaN,NaN,NaN,...,FBNUniversity,Hikma Pharmaceuticals,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
2,LUID_00003DAP,GB0004544929,454492,NaN,NaN,NaN,BBG000BYK740,NaN,NaN,NaN,...,fbn-uni-T02002,Imperial Brands,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
3,LUID_00003DC4,GB0002162385,216238,NaN,NaN,NaN,BBG000DDCSJ1,NaN,NaN,NaN,...,fbn-uni-T02002,Aviva,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
4,LUID_00003D5D,GB0008782301,NaN,NaN,TW/LN,imd_77645536,BBG000BF4KL1,NaN,NaN,NaN,...,FBNUniversityModule-1,Taylor Wimpey,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,LUID_00003D7G,GB0009895292,989529,NaN,NaN,NaN,BBG000C0YDQ1,NaN,NaN,NaN,...,fbn-uni-T02002,AstraZeneca,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
262,LUID_00003D59,GB00BLGZ9862,NaN,NaN,TSCO LN,imd_34634673,BBG000BF46Y8,NaN,NaN,NaN,...,FBNUniversityModule-1,Tesco,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
263,LUID_00003D56,GB0007980591,NaN,NaN,BP/LN,imd_43535553,BBG000C05BD1,NaN,NaN,NaN,...,FBNUniversityModule-1,BP,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
264,LUID_00003DHZ,IE0001827041,182704,NaN,NaN,NaN,BBG000BDCN13,NaN,NaN,NaN,...,FBNUniversity,CRH plc,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP


### JOINing disparate datasources with join
Now, I've got a file in drive with sector information for some of these instruments. I'm gonna join data from the file using our Drive provider with the saved instrument data so I can see the sectors for instruments where we know this information.

First lets get our instrument data.

Then we'll grab the file contents from drive.

We'll take a look at cross-joins, which Sqllite uses to build left joins and inner joins. Cross-joins return the cartesian product of both tables - each row on the left table is concatenated with every row on the right table, to form a massive table with every combination of both tables' rows:


In [13]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector FROM Lusid.Instrument.Equity as Equities CROSS JOIN @x as File

,DisplayName,DomCcy,Sector
0,Auto Trader Group,GBP,Financial Services
1,Auto Trader Group,GBP,General Industrials
2,Auto Trader Group,GBP,Travel & Leisure
3,Auto Trader Group,GBP,Equity Investment Instruments
4,Auto Trader Group,GBP,Household Goods & Home Construction
...,...,...,...
11795,Hikma Pharmaceuticals,GBP,Mining
11796,Hikma Pharmaceuticals,GBP,Media
11797,Hikma Pharmaceuticals,GBP,Mobile Telecommunications
11798,Hikma Pharmaceuticals,GBP,Food & Drug Retailers


In a left join, all rows from the left table are returned, along with rows in the right table that match the supplied key constraint. If there is a row in the left table with no matching row on the right, the columns that come from the right table in the resulting row are set to null.

We see this here as there are some companies with no matching sector in our file - these are returned as null.

In [14]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector FROM Lusid.Instrument.Equity as Equities LEFT JOIN @x as File on Equities.Figi = File.Figi

,DisplayName,DomCcy,Sector
0,BP,GBP,Oil & Gas Producers
1,Hikma Pharmaceuticals,GBP,Pharmaceuticals & Biotechnology
2,Sirius XM Holdings Inc.,USD,NaN
3,"Skyworks Solutions, Inc.",USD,NaN
4,Alphabet Inc. (Class A),USD,NaN
...,...,...,...
467,AstraZeneca,GBP,NaN
468,Tesco,GBP,NaN
469,BP,GBP,Oil & Gas Producers
470,CRH plc,GBP,NaN


We are going to ignore instruments without a matching sector for now, so we'll use an inner join.
Inner joins only return rows where the joining key has a match on both sides of the table:

In [15]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector FROM Lusid.Instrument.Equity as Equities INNER JOIN @x as File on Equities.Figi = File.Figi

,DisplayName,DomCcy,Sector
0,Segro,GBP,Real Estate Investment Trusts
1,BP,GBP,Oil & Gas Producers
2,Taylor Wimpey,GBP,Household Goods & Home Construction
3,Ferguson plc,GBP,Support Services
4,BP,GBP,Oil & Gas Producers
...,...,...,...
65,Glencore,GBP,Mining
66,BP,GBP,Oil & Gas Producers
67,Ferguson plc,GBP,Support Services
68,BP,GBP,Oil & Gas Producers


### GROUPing results BY some condition in query
We can also group our results by a column - we'll perform an aggregation and group-by to count the number of instrument in each sector.

In [16]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT count(File.Sector), File.Sector
FROM Lusid.Instrument.Equity as Equities 
Join @x as File on Equities.Figi = File.Figi
GROUP BY File.Sector

,count(File_Sector COLLATE BINARY),Sector
0,2,Automobiles & Parts
1,2,Equity Investment Instruments
2,4,Financial Services
3,2,Food & Drug Retailers
4,2,General Industrials
5,3,Household Goods & Home Construction
6,2,Investment Services
7,9,Life Insurance
8,2,Media
9,7,Mining


## Introducing some Luminesce extensions
### Using ^ to SELECT the most important columns from a provider
Let's use the carat symbol to grab then more important columns in the Equity provider. The list of important columns is decided by the writer of the Provider.

In [17]:
%%luminesce
SELECT ^ FROM Lusid.Instrument.Equity

,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive
0,LUID_00003D67,default,Auto Trader Group,Equity,Equities,GBP,True
1,LUID_00003D6C,default,BAE Systems,Equity,Equities,GBP,True
2,LUID_00003DBV,fbn-uni-T02002,Pennon Group,Equity,Equities,GBP,True
3,LUID_00003D72,fbn-uni-T02002,Broadcom Inc.,Equity,Equities,USD,True
4,LUID_00003D7V,fbn-uni-T02002,NXP Semiconductors,Equity,Equities,USD,True
...,...,...,...,...,...,...,...
467,LUID_00003DF1,FBNUniversity,"Skyworks Solutions, Inc.",Equity,Equities,USD,True
468,LUID_00003D5X,FBNUniversityModule-6-3,BP,Equity,Equities,GBP,True
469,LUID_00003DCY,FBNUniversity,Sirius XM Holdings Inc.,Equity,Equities,USD,True
470,LUID_00003D6I,default,British American Tobacco,Equity,Equities,GBP,True


### Excluding columns from the results of a query using the except keyword.
We can also exclude columns from the result of a query by using the except keyword:

In [18]:
%%luminesce
SELECT ^ (except Type, IsActive) FROM Lusid.Instrument.Equity

,LusidInstrumentId,Scope,DisplayName,InferredAssetClass,InferredDomCcy
0,LUID_00003D5K,FBNUniversityModule-3-1,BP,Equities,GBP
1,LUID_00003DE8,FBNUniversity,Hikma Pharmaceuticals,Equities,GBP
2,LUID_00003DCY,FBNUniversity,Sirius XM Holdings Inc.,Equities,USD
3,LUID_00003DF1,FBNUniversity,"Skyworks Solutions, Inc.",Equities,USD
4,LUID_00003DD4,FBNUniversity,Alphabet Inc. (Class A),Equities,USD
...,...,...,...,...,...
467,LUID_00003D7G,fbn-uni-T02002,AstraZeneca,Equities,GBP
468,LUID_00003D59,FBNUniversityModule-1,Tesco,Equities,GBP
469,LUID_00003D56,FBNUniversityModule-1,BP,Equities,GBP
470,LUID_00003DHZ,FBNUniversity,CRH plc,Equities,GBP


### Using parameters to filter queries
Many providers will have parameters that can also be used to filter a Luminesce query as part of a standard WHERE clause.

We can see these parameters by querying the sys.Field Provider.

In [19]:
%%luminesce
select FieldName, DataType, ParamDefaultValue, Description from Sys.Field where TableName = 'Lusid.Instrument.Equity' and FieldType = 'Parameter';

,FieldName,DataType,ParamDefaultValue,Description
0,AsAt,DateTime,0001-01-01T00:00:00Z,As at time to query at. Defaults to latest.
1,EffectiveAt,DateTime,0001-01-01T00:00:00Z,Effective time to query at. Defaults to latest.
2,UseLusidFilter,Boolean,True,Should the filter be translated to a Finbourne...


In [20]:
%%luminesce

select * from Lusid.Instrument.Equity where UseLusidFilter = False

,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,ClientInternal,Figi,CompositeFigi,ShareClassFigi,Wertpapier,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,LUID_00003D67,GB00BVYVFW23,BVYVFW2,NaN,NaN,NaN,BBG0086LKMG1,NaN,NaN,NaN,...,default,Auto Trader Group,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
1,LUID_00003D6C,GB0002634946,263494,NaN,NaN,NaN,BBG000BD5TW4,NaN,NaN,NaN,...,default,BAE Systems,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
2,LUID_00003DBV,GB00B18V8630,B18V863,NaN,NaN,NaN,BBG000H90Q58,NaN,NaN,NaN,...,fbn-uni-T02002,Pennon Group,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
3,LUID_00003D72,US11135F1012,NaN,11135F101,NaN,NaN,BBG00KHY5S69,NaN,NaN,NaN,...,fbn-uni-T02002,Broadcom Inc.,NaN,NaN,Equity,Equities,USD,True,NaN,USD
4,LUID_00003D7V,NL0009538784,NaN,NaN,NaN,NaN,BBG000BND699,NaN,NaN,NaN,...,fbn-uni-T02002,NXP Semiconductors,NaN,NaN,Equity,Equities,USD,True,NaN,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,LUID_00003DF1,US83088M1027,NaN,83088M102,NaN,NaN,BBG000GB4HW0,NaN,NaN,NaN,...,FBNUniversity,"Skyworks Solutions, Inc.",NaN,NaN,Equity,Equities,USD,True,NaN,USD
468,LUID_00003D5X,NaN,NaN,NaN,NaN,NaN,BBG000C05BD1,NaN,NaN,NaN,...,FBNUniversityModule-6-3,BP,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
469,LUID_00003DCY,US82968B1035,NaN,82968B103,NaN,NaN,BBG000BGPKZ1,NaN,NaN,NaN,...,FBNUniversity,Sirius XM Holdings Inc.,NaN,NaN,Equity,Equities,USD,True,NaN,USD
470,LUID_00003D6I,GB0002875804,287580,NaN,NaN,NaN,BBG000BG9N74,NaN,NaN,NaN,...,default,British American Tobacco,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP


## Using Luminesce variables
You can use variables to create and populate arbitrary tables of data that can then be used as part of a Luminesce query for either a data provider or a direct provider.

Declare a variable that represents a table of data using:
`@variable`

Or to represent a single value:
`@@variable`

Let's see variables in action. Here_instruments_in GBP is a table variable, set from the results of the query on equities.

In [21]:
%%luminesce
@instruments_in_GBP = select * from Lusid.Instrument.Equity where DomCcy = 'GBP';
Select Isin from @instruments_in_GBP;


,Isin
0,GB00B0LCW083
1,GB0004544929
2,GB0002162385
3,GB0008782301
4,GB0009895292
...,...
249,GB0009895292
250,GB00BLGZ9862
251,GB0007980591
252,IE0001827041


In [22]:
%%luminesce
@sectors = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

@@selected_sector = SELECT 'Life Insurance';
SELECT * FROM @sectors WHERE Sector = @@selected_sector

,Name,Sector,Figi
0,Prudential plc,Life Insurance,BBG000BDY322
1,St. James's Place plc,Life Insurance,BBG000BRMPC4
2,Legal & General,Life Insurance,BBG000BDQCG6
3,Aviva,Life Insurance,BBG000DDCSJ1


## Introducing more advanced Luminesce extensions
You can use the CROSS APPLY or OUTER APPLY statement to iterate over a set of parameters. This is useful to execute a Luminesce query in parallel for each of the parameter inputs.
### Cross-apply
Similar to INNER JOIN, in that only records that have matching values on both sides are returned. 
Here, we'll run some equity queries in parallel using Tools.split to create a set of currencies to filter by- the table will contain the currency joined to all of the returned equity tables:

In [23]:
%%luminesce
@currencies = select VALUE as val
from Tools.Split where Original = 'GBP USD CHF' AND Delimiters = ' ';

select c.val as selectedCurrency, results.*
from @currencies c
cross apply (
       SELECT equity.^ FROM Lusid.Instrument.Equity equity WHERE equity.DomCcy = c.val
       
) results;

,selectedCurrency,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,__row_key__
0,GBP,LUID_00003D67,default,Auto Trader Group,Equity,Equities,GBP,True,1
1,GBP,LUID_00003D6C,default,BAE Systems,Equity,Equities,GBP,True,1
2,GBP,LUID_00003DBV,fbn-uni-T02002,Pennon Group,Equity,Equities,GBP,True,1
3,GBP,LUID_00003DHZ,FBNUniversity,CRH plc,Equity,Equities,GBP,True,1
4,GBP,LUID_00003DHD,FBNUniversity,Segro,Equity,Equities,GBP,True,1
...,...,...,...,...,...,...,...,...,...
467,USD,LUID_00003D9J,fbn-uni-T02002,Marriott International,Equity,Equities,USD,True,2
468,USD,LUID_00003DAG,fbn-uni-T02002,Xcel Energy Inc.,Equity,Equities,USD,True,2
469,USD,LUID_00003D8O,fbn-uni-T02002,Marvell Technology Group,Equity,Equities,USD,True,2
470,USD,LUID_00003D8V,fbn-uni-T02002,Kraft Heinz Co,Equity,Equities,USD,True,2


### Outer-apply
Similar to LEFT OUTER JOIN, in that all records on the left side are returned, even when there are no matching records on the right side.
We'll see that the CHF record is returned even though there are no instruments that have a DomCcy with value CHF.

In [24]:
%%luminesce
@currencies = select VALUE as val
from Tools.Split where Original = 'GBP USD CHF' AND Delimiters = ' ';

select c.val, results.*
from @currencies c
outer apply (
       SELECT equity.^ FROM Lusid.Instrument.Equity equity WHERE equity.DomCcy = c.val
       
) results;

,val,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,__row_key__
0,GBP,LUID_00003D4V,FBNUniversityModule-1,RELX,Equity,Equities,GBP,True,1.0
1,GBP,LUID_00003D4W,FBNUniversityModule-1,Halma,Equity,Equities,GBP,True,1.0
2,GBP,LUID_00003D4X,FBNUniversityModule-1,Unilever,Equity,Equities,GBP,True,1.0
3,GBP,LUID_00003D4Y,FBNUniversityModule-1,Just Eat Takeawy,Equity,Equities,GBP,True,1.0
4,GBP,LUID_00003D4Z,FBNUniversityModule-1,Mondi,Equity,Equities,GBP,True,1.0
...,...,...,...,...,...,...,...,...,...
468,USD,LUID_00003DHY,FBNUniversity,JD.com,Equity,Equities,USD,True,2.0
469,USD,LUID_00003DI0,FBNUniversity,Fox Corporation (Class A),Equity,Equities,USD,True,2.0
470,USD,LUID_00003DI1,FBNUniversity,Electronic Arts,Equity,Equities,USD,True,2.0
471,USD,LUID_00003DI5,FBNUniversity,Keurig Dr Pepper Inc.,Equity,Equities,USD,True,2.0
